In [1]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
mpdr = MPDataRetrieval(api_key="TUvMnIDcCmqNEiG0")
from pymatgen.ext.matproj import MPRester
mpr = MPRester('TUvMnIDcCmqNEiG0')
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram
from collections import defaultdict

### testing the pourbaix diagram plot
# from pymatgen.analysis.pourbaix_diagram import PourbaixPlotter
# plotter = PourbaixPlotter(pbx)
# plotter.get_pourbaix_plot().show()

/opt/miniconda3/envs/py3/lib/python3.8/site-packages/pymatgen/ext/matproj.py:454: FutureWarning: __init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020
  def get_pourbaix_entries(self, chemsys, solid_compat=MaterialsProjectCompatibility()):


In [4]:
nonmetal = ["H", "He", "B", "C", "N", "F", "Ne", "Si", "P",
            "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
            "At", "Rn"]
nonmetal_wO = ["H", "He", "B", "C", "N", "O", "F", "Ne", "Si", "P",
            "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
            "At", "Rn"]

df_MO = mpdr.get_dataframe(criteria={"nelements": 2,"elements": {"$nin": nonmetal}, "elements": {"$all": ["O"]}},
                           properties=['material_id', 'pretty_formula', 'e_above_hull', 'formation_energy_per_atom'])
print("There are {} entries of metal oxides".format(df_MO['pretty_formula'].count()))

df_BM = mpdr.get_dataframe(criteria={"nelements": 2,"elements": {"$nin": nonmetal_wO}},
                           properties=['material_id', 'pretty_formula', 'e_above_hull', 'formation_energy_per_atom'])
print("There are {} entries of bimetallic compounds".format(df_BM['pretty_formula'].count()))

df_MM = mpdr.get_dataframe(criteria={"nelements": 1,"elements": {"$nin": nonmetal_wO}},
                           properties=['material_id', 'pretty_formula', 'e_above_hull'])
print("There are {} entries of monometallic compounds".format(df_MM['pretty_formula', 'formation_energy_per_atom'].count()))

  0%|          | 0/2333 [00:00<?, ?it/s]

There are 2333 entries of metal oxides


  0%|          | 0/9469 [00:00<?, ?it/s]

There are 9469 entries of bimetallic compounds


  0%|          | 0/14932 [00:00<?, ?it/s]

There are 14932 entries of trimetallic compounds


In [2]:
# nonmetal = ["H", "He", "B", "C", "N", "F", "Ne", "Si", "P",
#             "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
#             "At", "Rn"]
# nonmetal_wO = ["H", "He", "B", "C", "N", "O", "F", "Ne", "Si", "P",
#             "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
#             "At", "Rn"]
# df_MO = mpdr.get_dataframe(criteria={"nelements": 2,"elements": "$nin": nonmetal, "elements": {"$all": ["O"]}},
#                            properties=['material_id', 'pretty_formula', 'e_above_hull'])
# print("There are {} entries of metal oxides".format(df_MO['pretty_formula'].count()))

# df_BM = mpdr.get_dataframe(criteria={"nelements": 2,"elements": {"$nin": nonmetal_wO}},
#                            properties=['material_id', 'pretty_formula', 'e_above_hull'])
# print("There are {} entries of bimetallic compounds".format(df_BM['pretty_formula'].count()))

# df_TM = mpdr.get_dataframe(criteria={"nelements": 3,"elements": {"$nin": nonmetal_wO}},
#                             properties=['material_id', 'pretty_formula', 'e_above_hull'])
# print("There are {} entries of trimetallic compounds".format(df_TM['pretty_formula'].count()))

# df_MO.to_json('df_MO.json')
# df_BM.to_json('df_BM.json')
# df_TM.to_json('df_TM.json')

In [3]:
from pymatgen.core import Composition
def screen_metallic(mat_list, oxides=True):
    new_dict = defaultdict(dict)
    nonmetal = ["H", "He", "B", "C", "N", "F", "Ne", "Si", "P",
                "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
                "At", "Rn", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu",
                "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Ac", "Th", "Pa", "U", "Np", "Pu"]
    nonmetal_wO = ["H", "He", "B", "C", "N", "O", "F", "Ne", "Si", "P",
                   "S", "Cl", "Ar", "As", "Se", "Br", "Kr", "Te", "I", "Xe",
                   "At", "Rn", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu",
                   "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Ac", "Th", "Pa", "U", "Np", "Pu"]
    screen_list = nonmetal if oxides else nonmetal_wO
    for i in range(len(mat_list)):
        adding_tag = True
        temp_key = list(mat_list.keys())[i]
        temp_value = list(mat_list.values())[i]
        MP_composition = Composition(temp_value)
        for ele in (MP_composition.elements):
            if str(ele) in screen_list:
                adding_tag = False
        if (adding_tag):
            new_dict[temp_key] = temp_value
    return new_dict

In [4]:
import json
f_MO = open('df_MO.json', 'r')
MO = json.load(f_MO)
MO = MO['pretty_formula']
MO_list = screen_metallic(MO)
f_BM = open('df_BM.json', 'r')
BM = json.load(f_BM)
BM = BM['pretty_formula']
BM_list = screen_metallic(BM, oxides=False)
f_TM = open('df_TM.json', 'r')
TM = json.load(f_TM)
TM = TM['pretty_formula']
TM_list = screen_metallic(TM, oxides=False)

In [5]:
print (len(MO),len(BM),len(TM))
print (len(MO_list),len(BM_list),len(TM_list))

2333 9469 14932
1617 5589 6832


In [6]:
def get_phxeng(mat_dict, ph_max=7, pot_mmin=-1.0, ifMO=False):
    selected_dict = defaultdict(dict)
    for d in range(len(mat_dict)):
        if d % 51 == 0:
            mpr = MPRester('TUvMnIDcCmqNEiG0')
            from time import sleep
            sleep(10)
#         if d == 20: break
        temp_key = list(mat_dict.keys())[d]
        temp_value = list(mat_dict.values())[d]
        print (d, temp_value)
        temp_composition = Composition(temp_value)
        mpcomound = []
        for i in temp_composition.elements:
            mpcomound.append(str(i))
        if (ifMO):
            mpcomound.remove('O')
            entries = mpr.get_pourbaix_entries(mpcomound)
            print (mpcomound)
            pbx = PourbaixDiagram(entries)
        else:
            entries = mpr.get_pourbaix_entries(mpcomound)
            print (len (entries))
            composition_dict = temp_composition.fractional_composition.as_dict()
            pbx = PourbaixDiagram(
                entries,
                comp_dict=composition_dict,
                filter_solids=True,
            )
#             from pymatgen.analysis.pourbaix_diagram import PourbaixPlotter
#             plotter = PourbaixPlotter(pbx)
#             plotter.get_pourbaix_plot().show()
        entry = [e for e in entries if e.entry_id == temp_key][0]
        small_phx = 100
        for pH_val in range (0, ph_max+1):
            temp_phx = pbx.get_decomposition_energy(entry, pH=pH_val, V=pot_mmin)
            if temp_phx <= small_phx:
                small_phx = temp_phx
                temp_pH = pH_val
#           print (small_phx, temp_pH)
        selected_dict[temp_key]["composition"] = temp_value
        selected_dict[temp_key]["energy"] = small_phx
        print ("done")
    return selected_dict

In [7]:
def split_dict(d, n):
    keys = list(d.keys())
    for i in range(0, len(keys), n):
        yield {k: d[k] for k in keys[i: i + n]}
# for item in split_dict({i: i for i in range(10)}, 3):
#     print(item)

In [8]:
temp_store_dict = defaultdict(dict)

In [14]:
breakpoint = 13
i = -1
for phx_chunk in split_dict(MO_list, 100):
    i += 1
    print (i)
    if i < breakpoint: continue
    temp_store_dict[i] = get_phxeng(phx_chunk, ifMO=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
0 VO2
['V']
done
1 NbO2
['Nb']
done
2 Li2O
['Li']
done
3 Cu2O3
['Cu']
done
4 CoO2
['Co']
done
5 Fe13O15
['Fe']
done
6 Fe10O11
['Fe']
done
7 BeO
['Be']
done
8 Sc16O15
['Sc']
done
9 Cu8O7
['Cu']
done
10 Zr6O11
['Zr']
done
11 Co15O28
['Co']
done
12 Mn2O3
['Mn']
done
13 Mn2O3
['Mn']
done
14 V5O12
['V']
done
15 ZrO2
['Zr']
done
16 V2O5
['V']
done
17 K2O
['K']
done
18 HfO2
['Hf']
done
19 HfO2
['Hf']
done
20 BaO
['Ba']
done
21 V2O3
['V']
done
22 TiO2
['Ti']
done
23 Y2O3
['Y']
done
24 Cr19O48
['Cr']
done
25 V2O3
['V']
done
26 GeO2
['Ge']
done
27 Cs2O2
['Cs']
done
28 Cs2O
['Cs']
done
29 Al2O
['Al']
done
30 AlO
['Al']
done
31 Ti3O5
['Ti']
done
32 RuO2
['Ru']
done
33 V2O5
['V']
done
34 Mn21O40
['Mn']
done
35 FeO3
['Fe']
done
36 Fe32O35
['Fe']
done
37 Ga2O3
['Ga']
done
38 ZrO2
['Zr']
done
39 Ge2O3
['Ge']
done
40 K2O
['K']
done
41 CrO2
['Cr']
done
42 Bi4O9
['Bi']
done
43 VO2
['V']
done
44 FeO2
['Fe']
done
45 FeO2
['Fe']
done
46 CrO2
['Cr']
done
47 FeO2
['Fe']
done
48

In [11]:
print (i)

for key in temp_store_dict.keys():
    print (key)

13
9
10
11
12


In [12]:
# import json
# with open('temp_store_MO.json', 'w') as fp:
#     json.dump(temp_store_dict, fp)

# len(temp_store_dict)

In [13]:
f_prev_stored_MO = open('temp_store_MO.json', 'r')
prev_stored_MO = defaultdict(dict)
prev_stored_MO = json.load(f_prev_stored_MO)
prev_stored_MO

for key in temp_store_dict.keys():
    prev_stored_MO[key] = dict(temp_store_dict[key])

with open('temp_store_MO.json', 'w') as fp:
    json.dump(prev_stored_MO, fp)

len(prev_stored_MO)

13

In [12]:
with open('store_MO.json', 'w') as fp:
    json.dump(prev_stored_MO, fp)